In [6]:
import pandas as pd

df = pd.read_csv('.\soc-redditHyperlinks-title.tsv', sep='\t', header=0)

df

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES
0,rddtgaming,rddtrust,1u4pzzs,2013-12-31 16:39:18,1,"25.0,23.0,0.76,0.0,0.44,0.12,0.12,4.0,4.0,0.0,..."
1,xboxone,battlefield_4,1u4tmfs,2013-12-31 17:59:11,1,"100.0,88.0,0.78,0.02,0.08,0.13,0.07,16.0,16.0,..."
2,ps4,battlefield_4,1u4tmos,2013-12-31 17:59:40,1,"100.0,88.0,0.78,0.02,0.08,0.13,0.07,16.0,16.0,..."
3,fitnesscirclejerk,leangains,1u50xfs,2013-12-31 19:01:56,1,"49.0,43.0,0.775510204082,0.0,0.265306122449,0...."
4,fitnesscirclejerk,lifeprotips,1u51nps,2013-12-31 21:02:28,1,"14.0,14.0,0.785714285714,0.0,0.428571428571,0...."
...,...,...,...,...,...,...
571922,seattleshitshow,seattlewa,68iptp,2017-04-27 11:07:31,-1,"83.0,70.0,0.78313253012,0.0,0.10843373494,0.16..."
571923,peoplewhosayheck,spiderman,68iqh6s,2017-04-30 16:36:07,1,"121.0,105.0,0.719008264463,0.0165289256198,0.0..."
571924,vzla,wtf,68iqkcs,2017-04-30 15:50:07,1,"68.0,58.0,0.764705882353,0.0,0.147058823529,0...."
571925,denverbroncos,nfl,68iqt8,2017-04-30 16:11:30,-1,"96.0,79.0,0.770833333333,0.0,0.0729166666667,0..."


In [7]:
import itertools as it
import operator as op

d = {}

for i in range(len(df['SOURCE_SUBREDDIT'])):
    src = df['SOURCE_SUBREDDIT'][i]
    trg = df['TARGET_SUBREDDIT'][i]
    key = src, trg
    if key in d:
        v = d[key]
        v[2] = v[2] + 1
    else:
        d[key] = [src, trg, 1]

In [8]:
dval = d.values()

dval

In [9]:
df_out = pd.DataFrame({"SOURCE_SUBREDDIT":[sub[0] for sub in dval],
                     "TARGET_SUBREDDIT":[sub[1] for sub in dval],
                     "WEIGHT":[1/sub[2] for sub in dval]})
df_out

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,WEIGHT
0,rddtgaming,rddtrust,1.000000
1,xboxone,battlefield_4,1.000000
2,ps4,battlefield_4,0.250000
3,fitnesscirclejerk,leangains,0.045455
4,fitnesscirclejerk,lifeprotips,0.050000
...,...,...,...
234787,szechuansauceseekers,videos,1.000000
234788,nocontract,projectfi,1.000000
234789,peoplewhosayheck,dfsports,1.000000
234790,brasilonreddit,roadcam,1.000000


In [10]:
df_out.to_csv('.\soc-redditHyperlinks-title-parsed-fraction.csv', sep=",", index=False, header=False)

In [11]:
import networkx as nx

with open('.\soc-redditHyperlinks-title-parsed-fraction.csv') as myfile:
    head = [next(myfile) for x in range(3000)]
    
G = nx.read_weighted_edgelist(head, delimiter=",", create_using=nx.DiGraph())

In [133]:
fig, ax = plt.subplots(figsize=(50,50))

# pos = nx.spring_layout(G, k=1, iterations=1, scale=2)
nx.draw_shell(G, node_size=100)

In [19]:
import statistics
from itertools import chain

path_lengths = (x.values() for x in dict(nx.shortest_path_length(G)).values())
statistics.mean(chain.from_iterable(path_lengths))

5.884099715514577

In [23]:
nx.shortest_path_length(G, target='askreddit')

In [12]:
nx.dijkstra_path_length(G, 'ps4', 'battlefield_4')

0.25

In [24]:
import statistics
from itertools import chain

path_lengths = (dist.values() for (dist, path) in dict(nx.all_pairs_dijkstra(G)).values())
statistics.mean(chain.from_iterable(path_lengths))

1.7970507986464566

In [82]:
dict(nx.all_pairs_dijkstra(G))['fitnesscirclejerk'][1]

In [26]:
largest = max(nx.strongly_connected_components(G), key=len)

largest

In [45]:
for c in sorted(nx.strongly_connected_components(G), key=len, reverse=True):
    print(len(c), c)

In [41]:
for c in sorted(nx.weakly_connected_components(G), key=len, reverse=True):
    if (len(c) < 10):
        print(len(c), c)
    else:
        print(len(c), '{...}')

In [50]:
nodes_num = len(G.nodes())
strong = len(max(nx.strongly_connected_components(G), key=len))
weak = len(max(nx.weakly_connected_components(G), key=len))

print('Strongly connected: ', strong / nodes_num)
print('Weakly connected: ', weak / nodes_num)

Strongly connected:  0.05620608899297424
Weakly connected:  0.7480093676814988


In [52]:
import statistics
from itertools import chain

path_lengths = (x.values() for x in dict(nx.shortest_path_length(G)).values())
mean = statistics.mean(chain.from_iterable(path_lengths))
path_lengths_weighted = (dist.values() for (dist, path) in dict(nx.all_pairs_dijkstra(G)).values())
mean_weighted = statistics.mean(chain.from_iterable(path_lengths_weighted))

print('The mean shortest path length of the network: ', mean)
print('The mean weighted shortest path length of the network: ', mean_weighted)

The mean shortest path length of the network:  5.884099715514577
The mean weighted shortest path length of the network:  1.7970507986464566


In [107]:
shortest_path_lengths = dict(nx.all_pairs_dijkstra(G, weight=None))
data = {}

for key in shortest_path_lengths:
    data[key] = {}
    for target in shortest_path_lengths[key][1]:
        data[key][target] = {}
        data[key][target]['weight'] = 0
        path = shortest_path_lengths[key][1][target]
        data[key][target]['dist'] = shortest_path_lengths[key][0][target]
        for i in range(len(path)):
            if i == 0:
                edge = G.get_edge_data(key, path[i])
                if edge is not None:
                    data[key][target]['weight'] = data[key][target]['weight'] + edge['weight']
            else:
                edge = G.get_edge_data(path[i-1], path[i])
                if edge is not None:
                    data[key][target]['weight'] = data[key][target]['weight'] + edge['weight']

data['languagelearning']['technology']

{'weight': 1.3160597572362278, 'dist': 4}

In [128]:
import statistics
from itertools import chain

path_lengths = []
path_weights = []
for targets in weight.values():
    for target in targets:
        path_weights.append(targets[target]['weight'])
        path_lengths.append(targets[target]['dist'])
        
mean = statistics.mean(path_lengths)
mean_weighted = statistics.mean(path_weights)

print('The mean shortest path length of the network: ', mean)
print('The mean weigh of these paths: ', mean_weighted)

The mean shortest path length of the network:  5.884099715514577
The mean weigh of these paths:  2.666556564286347


In [31]:
len(G.nodes())

2135

In [20]:
from igraph import *

def parse_string(x, max_len=100):
    out = 'NONE'
    try:
        out = str(x.encode('ascii'))[:max_len]
    except:
        pass
    return out


def read_file(
    filename,
    directed=True,
    etype_names=None,
    header=False,
    ):

    edgelist = []
    etypes = []
    names = UniqueIdGenerator()
    header_info = []
    with open(filename) as fp:
        data = fp.readlines()
        for line in data:
            line = line.strip().split(',')
            if header:
                header_info = line
                header = False
                continue
            n1 = names[line[0]]
            n2 = names[line[1]]
            edgelist.append((n1, n2))
            if etype_names is not None and len(line) \
                == len(etype_names) + 2:
                etypes.append(tuple(line[2:]))
    g = Graph(edgelist, directed=directed)
    g.vs['name'] = [names.reverse_dict()[vid.index] for vid in g.vs]
    g.vs['label'] = [parse_string(v) for v in g.vs['name']]
    if etype_names is not None:
        for (k, v) in zip(etype_names, zip(*etypes)):
            g.es[k] = v
    return g


etype_names = ['weight']
g = read_file('soc-redditHyperlinks-title-parsed.csv', etype_names=etype_names, directed=False,
              header=False)

g.summary()

'IGRAPH UNW- 54075 234792 -- \n+ attr: label (v), name (v), weight (e)'

In [2]:
visual_style = {}
visual_style["vertex_size"] = 20
visual_style["vertex_label"] = g.vs["name"]
visual_style["layout"] = g.layout("kk")
visual_style["bbox"] = (300, 300)
visual_style["margin"] = 20

plot(g, "result.pdf", visual_style)

NameError: name 'g' is not defined